In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install catboost

In [ ]:
!pip install catboost

     |████████████████████████████████| 64.8MB 59kB/s 


In [ ]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor, VotingRegressor
from sklearn.linear_model import BayesianRidge, LinearRegression, LassoLars, Ridge

# import sweetviz as sv

import matplotlib.pyplot as plt

import seaborn as sns

from xgboost import XGBRegressor

from catboost import CatBoostRegressor

from lightgbm import LGBMRegressor

import tqdm

import optuna

# from autofeat import AutoFeatRegressor

# from pystacknet.pystacknet import StackNetRegressor

# from GML.Ghalat_Machine_Learning import Ghalat_Machine_Learning

ModuleNotFoundError: ignored

In [ ]:
def accuracy(y_true,y_pred):
  return 100 * max(0,1-np.sqrt(mean_squared_error(y_true,y_pred)))

In [ ]:
path = 'drive/My Drive/Hackerearth/ML1/Dataset/'

In [ ]:
train = pd.read_csv(path+'Train.csv')
test = pd.read_csv(path+'Test.csv')

In [ ]:
test_eid = test['Employee_ID'].copy()

In [ ]:
train['Employee_ID'] = train['Employee_ID'].apply(lambda x: int(x.split('_')[1]))
test['Employee_ID'] = test['Employee_ID'].apply(lambda x: int(x.split('_')[1]))

In [ ]:
train.groupby('Post_Level')['Attrition_rate'].mean()

Post_Level
1    0.189832
2    0.185217
3    0.187232
4    0.195130
5    0.198197
Name: Attrition_rate, dtype: float64

In [ ]:
avg_att_dic = dict(train.groupby('Travel_Rate')['Attrition_rate'].mean())

In [ ]:
def avg_att(var):
    try:
        return avg_att_dic(var)
    except:
        return train['Attrition_rate'].mean()
    
def growth_rate(s):
    if s['growth_rate'] <= 30:
        return 'Low'
    elif s['growth_rate'] <= 55:
        return 'Medium'
    else:
        return 'High'

In [ ]:
tr1 = train['Travel_Rate'].apply(avg_att)
te1 = test['Travel_Rate'].apply(avg_att)

In [ ]:
train['growth_rate_level'] = train.apply(growth_rate,axis=1)
test['growth_rate_level'] = train.apply(growth_rate,axis=1)

In [ ]:
grid_lgbm = {
    'max_depth':16,
    'learning_rate':0.002,
    'n_estimators':1704,
    'min_child_weight':17,
    'eta':0.364
}

In [ ]:
cols = train.columns[train.dtypes == 'object']


for col in cols:
  dummies = pd.get_dummies(train[col])
  train.drop([col],axis=1,inplace=True)
  train = pd.concat([train,dummies],axis=1)
  dummies = pd.get_dummies(test[col])
  test.drop([col],axis=1,inplace=True)
  test = pd.concat([test,dummies],axis=1)

"""
for col in cols:
    encoder = LabelEncoder()
    train[col] = encoder.fit_transform(train[col])
    test[col] = encoder.transform(test[col])
"""

for col in test.columns:
    if train[col].isnull().any():
        model = LGBMRegressor(**grid_lgbm)
        t1 = train[~pd.isna(train[col])].copy()
        t1.drop(['Attrition_rate'],axis=1,inplace=True)
        t2 = train[pd.isna(train[col])].copy()
        t2.drop(['Attrition_rate','Age'],axis=1,inplace=True)
        t3 = test[pd.isna(test[col])].copy()
        t3.drop(['Age'],axis=1,inplace=True)
        model.fit(t1.drop([col],axis=1),t1[col])
        train.loc[t2.index,col] = model.predict(t2)
        test.loc[t3.index,col] = model.predict(t3)

In [ ]:
grid_bayesian = {
    'n_iter':10000,
    'tol':6.42e-06,
    'alpha_1':0.0001,
    'alpha_2':6.58e-05,
    'lambda_1':0.999,
    'lambda_2':2.24e-05,
    'compute_score':True,
    'fit_intercept':True,
    'normalize':False
}

grid_ridge = {
    'max_iter':90000,
    'alpha': 0.23143352151759083, 
    'fit_intercept': True, 
    'normalize': False, 
    'tol': 0.0005243805101392493, 
    'solver': 'saga'
}

In [ ]:
X = train.drop(['Attrition_rate'],axis=1)
y = train['Attrition_rate'].copy().values

In [ ]:
lgbm = LGBMRegressor(**grid_lgbm)
lgbm.fit(X, y)

fi = pd.DataFrame()
fi['A'] = X.columns.values
fi['B'] = lgbm.feature_importances_

In [ ]:
fi

,A,B
0,Employee_ID,5981
1,Age,3898
2,Education_Level,1430
3,Time_of_service,4166
4,Time_since_promotion,1173
5,growth_rate,5204
6,Travel_Rate,1113
7,Post_Level,1987
8,Pay_Scale,2546
9,Work_Life_balance,1611


In [ ]:
br = BayesianRidge(**grid_bayesian)
ridge = Ridge(**grid_ridge)

In [ ]:
scores = []
for tr_in, val_in in tqdm.tqdm(KFold().split(X, y)):
    br.fit(X.iloc[tr_in], y[tr_in])
    ridge.fit(X.iloc[tr_in], y[tr_in])
    scores.append(accuracy(y[val_in],
                           br.predict(X.iloc[val_in])*0.9 + 
                           ridge.predict(X.iloc[val_in])*0.1))

5it [00:18,  3.73s/it]


In [ ]:
def tunning(trial):
  learning_rate = trial.suggest_loguniform('learning_rate',1e-6,1)
  depth = trial.suggest_int('depth',2,60)
  

In [ ]:
scores = []
for tr_in, val_in in tqdm.tqdm(KFold().split(X, y)):
    cat = CatBoostRegressor(n_estimators = 1000, verbose=0)
    cat.fit(X.iloc[tr_in], y[tr_in])
    scores.append(accuracy(y[val_in],cat.predict(X.iloc[val_in])))

5it [00:17,  3.50s/it]


In [ ]:
scores

[81.95252387473838,
 82.02522250453244,
 81.30114960035479,
 81.77319159384338,
 80.15184054858985]

In [ ]:
np.mean(scores)

81.44078562441176

In [ ]:
preds = cat.predict(test)
ss = pd.DataFrame()
ss['Employee_ID'] = test_eid
ss['Attrition_rate'] = preds

ss.to_csv(path+'submission.csv',index=False)

In [ ]:
ss

,Employee_ID,Attrition_rate
0,EID_22713,0.197700
1,EID_9658,0.223686
2,EID_22203,0.154272
3,EID_7652,0.156179
4,EID_6516,0.204124
...,...,...
2995,EID_22547,0.230193
2996,EID_10066,0.143588
2997,EID_7126,0.180795
2998,EID_4929,0.331349
